# Topic Modelling for News

![](https://images.unsplash.com/photo-1495020689067-958852a7765e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

Photo by [Roman Kraft](https://unsplash.com/photos/_Zua2hyvTBk)

This exercise is about modelling the main topics of a database of News headlines.

Begin by importing the needed libraries:

In [1]:
# TODO: import needed libraries
import nltk
import numpy as np
import pandas as pd

Load the data in the file `random_headlines.csv`

In [3]:
# TODO: load the dataset
df = pd.read_csv('random_headlines.csv')
print(df.shape)
df.head()

(20000, 2)


,publish_date,headline_text
0,20120305,ute driver hurt in intersection crash
1,20081128,6yo dies in cycling accident
2,20090325,bumper olive harvest expected
3,20100201,replica replaces northernmost sign
4,20080225,woods targets perfect season


This is always a good idea to perform some EDA (exploratory data analytics) on a dataset...

In [4]:
# TODO: Perform a short EDA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   publish_date   20000 non-null  int64 
 1   headline_text  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


Now perform all the needed preprocessing on those headlines: case lowering, tokenization, punctuation removal, stopwords removal, stemming/lemmatization.

In [6]:
# TODO: Preprocess the input data

#tokenize
df["tokens"] = df["headline_text"].apply(lambda row: nltk.word_tokenize(row))

#pinctuation
df["alphanumeric"] = df["tokens"].apply(lambda row: [word for word in row if word.isalpha()])

#remove stopwords
stop = nltk.corpus.stopwords.words('english')
df["stop"] = df["alphanumeric"].apply(lambda row: [word for word in row if word not in stop])

#stemming
stemmer = nltk.PorterStemmer()
df["stemmed"] = df["stop"].apply(lambda row: [stemmer.stem(word) for word in row])

df["stemmed"].head()

0    [ute, driver, hurt, intersect, crash]
1                       [die, cycl, accid]
2          [bumper, oliv, harvest, expect]
3    [replica, replac, northernmost, sign]
4          [wood, target, perfect, season]
Name: stemmed, dtype: object

Now use Gensim to compute a BOW

In [9]:
# TODO: Compute the BOW using Gensim
from gensim.corpora import Dictionary

dictionary = Dictionary(df["stemmed"])
corpus = [dictionary.doc2bow(line) for line in df["stemmed"]]
print(np.shape(corpus))
corpus[0:2]

(20000,)


/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(5, 1), (6, 1), (7, 1)]]

In [7]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.0 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 5.2 MB/s eta 0:00:01


Compute the TF-IDF using Gensim

In [10]:
# TODO: Compute TF-IDF
from gensim.models import TfidfModel

tfidf_model = TfidfModel(corpus)
tf_idf = tfidf_model[corpus]
print(np.shape(tf_idf))

(20000,)


Finally compute the **LSA** (also called LSI) using Gensim, for a given number of Topics that you choose yourself

In [11]:
# TODO: Compute LSA
from gensim.models import LsiModel

lsi = LsiModel(corpus=corpus, num_topics=4, id2word=dictionary)

For each of the topic, show the most significant words.

In [13]:
# TODO: Print the 3 or 4 most significant words of each topic
lsi.print_topics(num_words=4)

[(0, '-0.752*"polic" + -0.405*"man" + -0.207*"charg" + -0.133*"new"'),
 (1, '0.668*"man" + -0.575*"polic" + 0.329*"charg" + 0.167*"court"'),
 (2, '-0.656*"new" + -0.296*"plan" + -0.241*"say" + 0.241*"man"'),
 (3, '-0.701*"new" + 0.345*"say" + 0.335*"plan" + 0.268*"govt"')]

What do you think about those results?

Now let's try to use LDA instead of LSA using Gensim

In [14]:
# TODO: Compute LDA
from gensim.models import LdaModel

lda = LdaModel(corpus=corpus, num_topics=4, id2word=dictionary)

In [15]:
# TODO: print the most frequent words of each topic
lda.print_topics(num_words=5)

[(0, '0.011*"interview" + 0.009*"win" + 0.005*"busi" + 0.005*"second"'),
 (1, '0.014*"man" + 0.010*"polic" + 0.009*"charg" + 0.009*"court"'),
 (2, '0.009*"polic" + 0.006*"kill" + 0.005*"murder" + 0.005*"plan"'),
 (3, '0.009*"new" + 0.007*"govt" + 0.006*"call" + 0.005*"report"')]

In [21]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 1.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136900 sha256=0deb7a80c87451eb085c3a4e3e1baee3ccdb33602ec33c5cf62514d12f1900dd
  Stored in directory: /Users/JasonWu/Library/Caches/pip/wheels/57/a4/86/d10c6c2e0bf149fbc0afb0aa5a6528ac35b30a133a0270c477
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=0711acab224eb53ac8ffffd74a7be2d738d005440b903cdafb22d3c77c6c970c
  Stored in directory: /Users/JasonWu/Library/Caches/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built pyLDAvis sklearn


Now, how does it work with LDA?

Let's make some visualization of the LDA results using pyLDAvis.

In [23]:
# TODO: show visualization results of the LDA
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
vis


/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: t

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.079887 -0.025716       1        1  26.803633
3     -0.018082 -0.194917       2        1  26.359923
1     -0.129154  0.156310       3        1  24.349152
0      0.227122  0.064323       4        1  22.487291, topic_info=          Term        Freq       Total Category  logprob  loglift
130  interview  283.000000  283.000000  Default  30.0000  30.0000
139        man  466.000000  466.000000  Default  29.0000  29.0000
312      court  289.000000  289.000000  Default  28.0000  28.0000
500        win  260.000000  260.000000  Default  27.0000  27.0000
86       charg  341.000000  341.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
340       open   69.974332  173.030065   Topic4  -5.7982   0.5869
439    council   77.681818  330.051560   Topic4  -5.6937   0.0456
520       take   68.271461  225.505887   Topic4  -5.8228   0.2974
243    concern   60.901542  154.976854   Topic4  -5.9370   0.5582
27       group   60.888505  157.384435   Topic4  -5.9373   0.5426

[315 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1103      1  0.047845   abbott
1103      2  0.940961   abbott
1103      3  0.015948   abbott
496       1  0.035704      abc
496       2  0.160667      abc
...     ...       ...      ...
255       2  0.981216    young
255       3  0.022300    young
2724      1  0.020642    youth
2724      2  0.970197    youth
3465      2  0.979033  zealand

[610 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1])

Depending on your results, you can try to fine tune the algorithm: number of topics, hyperparameters...
And check with others their results.